bài 1)

In [ ]:
import cv2
import numpy as np
import os
import random
from scipy.fft import fft2, ifft2, fftshift, ifftshift
from scipy.ndimage import minimum_filter, maximum_filter

# Thư mục ảnh gốc và đầu ra
folder_path = 'exercise'
output_path = 'output_rgb_butterworth'
os.makedirs(output_path, exist_ok=True)

# ================== Hàm biến đổi ảnh ==================

def shuffle_rgb_channels(img):
    channels = list(cv2.split(img))
    random.shuffle(channels)
    return cv2.merge(channels)

def butterworth_lowpass_filter(img, cutoff=30, order=2):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    dft = fftshift(fft2(gray))
    rows, cols = gray.shape
    crow, ccol = rows // 2, cols // 2

    u = np.arange(rows)
    v = np.arange(cols)
    U, V = np.meshgrid(u - crow, v - ccol, indexing='ij')
    D = np.sqrt(U**2 + V**2)
    H = 1 / (1 + (D / cutoff)**(2 * order))

    filtered = dft * H
    img_back = np.abs(ifft2(ifftshift(filtered)))
    return np.uint8(img_back)

def butterworth_highpass_filter(img, cutoff=30, order=2):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    dft = fftshift(fft2(gray))
    rows, cols = gray.shape
    crow, ccol = rows // 2, cols // 2

    u = np.arange(rows)
    v = np.arange(cols)
    U, V = np.meshgrid(u - crow, v - ccol, indexing='ij')
    D = np.sqrt(U**2 + V**2)
    H = 1 / (1 + (cutoff / D)**(2 * order))
    H[D == 0] = 0  # tránh chia cho 0

    filtered = dft * H
    img_back = np.abs(ifft2(ifftshift(filtered)))
    return np.uint8(img_back)

def min_filter(img, size=3):
    return minimum_filter(img, size=size)

def max_filter(img, size=3):
    return maximum_filter(img, size=size)

# ================== Xử lý ảnh ==================

def process_images():
    filters = ['lowpass', 'highpass']
    
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path)

            # Đổi thứ tự RGB
            shuffled_img = shuffle_rgb_channels(img)

            # Ngẫu nhiên chọn loại filter
            selected = random.choice(filters)

            if selected == 'lowpass':
                print(f"{file}: Butterworth Lowpass + Min Filter")
                result = butterworth_lowpass_filter(shuffled_img)
                result = min_filter(result)
            else:
                print(f"{file}: Butterworth Highpass + Max Filter")
                result = butterworth_highpass_filter(shuffled_img)
                result = max_filter(result)

            output_file = os.path.join(output_path, f'filtered_{file}')
            cv2.imwrite(output_file, result)
            cv2.imshow(f'Transformed - {file}', result)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

process_images()


colorful-ripe-tropical-fruits.jpg: Butterworth Lowpass + Min Filter
ha-long-bay-in-vietnam.jpg: Butterworth Highpass + Max Filter


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12088\2939733311.py:46: RuntimeWarning: divide by zero encountered in divide
  H = 1 / (1 + (cutoff / D)**(2 * order))


pagoda.jpg: Butterworth Highpass + Max Filter
quang_ninh.jpg: Butterworth Highpass + Max Filter


bài 2)

In [ ]:
import cv2
import os

# Đường dẫn ảnh
img_path = 'exercise/colorful-ripe-tropical-fruits.jpg'
img = cv2.imread(img_path)

# Kiểm tra ảnh
if img is None:
    print("Không tìm thấy ảnh.")
    exit()

# Sao chép ảnh để vẽ
output = img.copy()

# === BƯỚC 1: Cắt ROI thủ công (tọa độ cần chỉnh tay theo từng ảnh) ===
# Giả sử quả đu đủ nằm ở vùng (x1,y1,x2,y2), quả dưa hấu ở vùng khác
# Bạn cần tinh chỉnh 4 tọa độ này cho đúng ảnh thật của bạn!

# ROI quả đu đủ (ví dụ)
papaya_roi = img[50:200, 100:250]  # y1:y2 , x1:x2
# ROI quả dưa hấu (ví dụ)
watermelon_roi = img[180:300, 280:400]

# === BƯỚC 2: Đổi màu (đảo màu / chỉnh RGB) ===
papaya_roi = cv2.cvtColor(papaya_roi, cv2.COLOR_BGR2HSV)
papaya_roi[:, :, 0] = (papaya_roi[:, :, 0] + 60) % 180  # đổi tông màu
papaya_roi = cv2.cvtColor(papaya_roi, cv2.COLOR_HSV2BGR)

watermelon_roi = cv2.cvtColor(watermelon_roi, cv2.COLOR_BGR2HSV)
watermelon_roi[:, :, 0] = (watermelon_roi[:, :, 0] + 90) % 180
watermelon_roi = cv2.cvtColor(watermelon_roi, cv2.COLOR_HSV2BGR)

# === BƯỚC 3: Gắn lại vùng ảnh đã đổi màu vào ảnh gốc ===
output[50:200, 100:250] = papaya_roi
output[180:300, 280:400] = watermelon_roi

# === BƯỚC 4: Hiển thị và lưu ảnh ===
print("✅ Kết quả đã được lưu vào thư mục 'output_rotated'. Đang hiển thị ảnh kết quả...")
cv2.imshow("Original", img)
cv2.imshow("Modified", output)
cv2.imwrite("output_fruits_modified.jpg", output)
cv2.waitKey(0)
cv2.destroyAllWindows()


✅ Kết quả đã được lưu vào thư mục 'output_rotated'. Đang hiển thị ảnh kết quả...


bài 3)

In [ ]:
import cv2
import os
import numpy as np

# Đọc ảnh gốc
img_path = 'exercise/quang_ninh.jpg'
img = cv2.imread(img_path)

if img is None:
    print("Không tìm thấy ảnh.")
    exit()

# === BƯỚC 1: Cắt vùng đối tượng ===
# (Tọa độ ví dụ, bạn nên thay bằng tọa độ thực tế cho đúng ảnh)
# Format: img[y1:y2, x1:x2]

# Ngọn núi (ví dụ)
mountain = img[100:300, 150:400]

# Con thuyền (ví dụ)
boat = img[350:450, 500:600]

# === BƯỚC 2: Hàm xoay ảnh 1 góc bất kỳ ===
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    cos = np.abs(matrix[0, 0])
    sin = np.abs(matrix[0, 1])

    # Kích thước mới sau xoay
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))

    # Điều chỉnh lại tâm
    matrix[0, 2] += (new_w / 2) - center[0]
    matrix[1, 2] += (new_h / 2) - center[1]

    rotated = cv2.warpAffine(image, matrix, (new_w, new_h), borderValue=(255,255,255))
    return rotated

# === BƯỚC 3: Xoay 45 độ ===
mountain_rotated = rotate_image(mountain, 45)
boat_rotated = rotate_image(boat, 45)

# === BƯỚC 4: Lưu kết quả ===
output_folder = 'output_rotated'
os.makedirs(output_folder, exist_ok=True)

cv2.imwrite(os.path.join(output_folder, 'mountain_rotated.jpg'), mountain_rotated)
cv2.imwrite(os.path.join(output_folder, 'boat_rotated.jpg'), boat_rotated)

# Hiển thị kết quả
print("✅ Kết quả đã được lưu vào thư mục 'output_rotated'. Đang hiển thị ảnh kết quả...")
cv2.imshow("Mountain Rotated", mountain_rotated)
cv2.imshow("Boat Rotated", boat_rotated)
cv2.waitKey(0)
cv2.destroyAllWindows()



✅ Kết quả đã được lưu vào thư mục 'output_rotated'. Đang hiển thị ảnh kết quả...


bài 4)

In [ ]:
import cv2
import os

# Đọc ảnh từ thư mục
img_path = 'exercise/pagoda.jpg'
img = cv2.imread(img_path)

if img is None:
    print("Không tìm thấy ảnh.")
    exit()

# === BƯỚC 1: Cắt vùng ngôi chùa ===
#  Cần chỉnh tọa độ đúng với vùng chứa ngôi chùa trong ảnh của bạn
pagoda_roi = img[100:300, 150:350]  

# === BƯỚC 2: Phóng to 5 lần ===
scale_factor = 5
height, width = pagoda_roi.shape[:2]
resized = cv2.resize(pagoda_roi, (width * scale_factor, height * scale_factor), interpolation=cv2.INTER_CUBIC)

# === BƯỚC 3: Lưu và hiển thị ảnh kết quả ===
output_folder = 'output_pagoda'
os.makedirs(output_folder, exist_ok=True)

output_path = os.path.join(output_folder, 'pagoda_resized.jpg')
cv2.imwrite(output_path, resized)

print("✅ Đã lưu ảnh phóng to ngôi chùa vào:", output_path)
cv2.imshow("Ngôi chùa sau khi phóng to", resized)
cv2.waitKey(0)
cv2.destroyAllWindows()
